In [1]:
import os

import numpy as np
import pandas as pd

import torch
import torchvision

import matplotlib.pyplot as plt
import lightning as L

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.model_selection import train_test_split


c:\Users\1001002996\AppData\Local\miniconda3\Lib\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")


In [2]:
TEST = True

random_seed = 42
L.seed_everything(random_seed)

Seed set to 42


42

In [3]:
TRAIN_SIZE, VAL_SIZE, TEST_SIZE = 0.8, 0.1, 0.1

## TRAIN TEST SPLIT

In [4]:
dataset_path = os.path.join("dataset", "processed_concat_data")

In [5]:
fft_file = os.path.join(dataset_path, "torso_fft.npy")
label_file = os.path.join(dataset_path, "torso_label.npy")

In [6]:
prefix = "torso_"
postfix = "_fft"

fft_train_file = os.path.join(dataset_path, prefix + "train" + postfix + ".npy")
label_train_file = os.path.join(dataset_path, prefix + "train_label.npy")

fft_val_file = os.path.join(dataset_path, prefix + "val" + postfix + ".npy")
label_val_file = os.path.join(dataset_path, prefix + "val_label.npy")

fft_test_file = os.path.join(dataset_path, prefix + "test" + postfix + ".npy")
label_test_file = os.path.join(dataset_path, prefix + "test_label.npy")


In [7]:
fft_data = np.load(fft_file)
label = np.load(label_file)

In [8]:
print(np.unique(label, return_counts=True))

(array([1., 2., 3., 4., 5.]), array([26036, 26000,  8690, 24399, 33686], dtype=int64))


In [9]:
fft_data_down_samp_list = []
label_samp_list = []

for i in range(1, 1+5):
    one_class_label = label[label == i]
    choice_idx_list = np.random.choice(
        len(one_class_label), 
        min(10000, len(one_class_label)), 
        replace=False)
    
    label_samp_list.append(one_class_label[choice_idx_list])

    one_class_fft = fft_data[label == i]
    fft_data_down_samp_list.append(one_class_fft[choice_idx_list])

fft_data_down_samp = np.concatenate(fft_data_down_samp_list, axis=0)
label_samp = np.concatenate(label_samp_list, axis=0)

In [11]:
print(fft_data_down_samp.shape, label_samp.shape)

(48690, 6, 257) (48690,)


In [13]:

train_val_data, test_data, train_val_label, test_label = \
    train_test_split(fft_data_down_samp, label_samp, test_size=TEST_SIZE, stratify=label_samp, shuffle=True)

train_data, val_data, train_label, val_label = \
    train_test_split(train_val_data, train_val_label, test_size=VAL_SIZE / (TRAIN_SIZE + VAL_SIZE), stratify=train_val_label, shuffle=True)

print("train_data.shape, train_label.shape", train_data.shape, train_label.shape)
print("val_data.shape, val_label.shape", val_data.shape, val_label.shape)
print("test_data.shape, test_label.shape", test_data.shape, test_label.shape)

train_data.shape, train_label.shape (38952, 6, 257) (38952,)
val_data.shape, val_label.shape (4869, 6, 257) (4869,)
test_data.shape, test_label.shape (4869, 6, 257) (4869,)


In [14]:
np.save(fft_train_file, train_data)
np.save(label_train_file, train_label)
np.save(fft_val_file, val_data)
np.save(label_val_file, val_label)
np.save(fft_test_file, test_data)
np.save(label_test_file, test_label)